# Setup paths

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip -q install wget

  Preparing metadata (setup.py) ... done


In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks/TFODCourse - Copy'

/content/drive/.shortcut-targets-by-id/11Nbvfv2gDXwVgOrqKSOW69hmt_-zSksQ/TFODCourse - Copy


In [ ]:
import os
import wget
import glob
import tensorflow as tf
import xml.etree.ElementTree as ET

In [ ]:
# !rm -rf '/content/drive/MyDrive/Colab Notebooks/TFODCourse - Copy/Tensorflow/workspace/models/drone_ssd_mobnet'

In [ ]:
# define global variables
CUSTOM_MODEL_NAME = 'drone_ssd_mobnet_v4'
# http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = f'http://download.tensorflow.org/models/object_detection/tf2/20200711/{PRETRAINED_MODEL_NAME}.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map_2.pbtxt'
BATCH_SIZE = 4

In [ ]:
paths = {
    "WORKSPACE_PATH" : os.path.join('Tensorflow', 'workspace'),
    "SCRIPTS_PATH" : os.path.join('Tensorflow', 'scripts'),
    "APIMODEL_PATH" : os.path.join('Tensorflow', 'models'),
    "ANNOTATION_PATH" : os.path.join('Tensorflow', 'workspace', 'annotations'),
    "IMAGE_PATH" : os.path.join('Tensorflow', 'workspace', 'images'),
    "MODEL_PATH" : os.path.join('Tensorflow', 'workspace', 'models'),
    "PRETRAINED_MODEL_PATH" : os.path.join('Tensorflow', 'workspace', 'pre-trained-models'),
    "CHECKPOINT_PATH" : os.path.join('Tensorflow', 'workspace', 'models', CUSTOM_MODEL_NAME),
    "OUTPUT_PATH" : os.path.join('Tensorflow', 'workspace', 'models', CUSTOM_MODEL_NAME, 'export'),
    "TFJS_PATH" : os.path.join('Tensorflow', 'workspace', 'models', CUSTOM_MODEL_NAME, 'tfjsexport'),
    "TFLITE_PATH" : os.path.join('Tensorflow', 'workspace', 'models', CUSTOM_MODEL_NAME, 'tfliteexport'),
    "PROTOC_PATH" : os.path.join('Tensorflow', 'protoc'),
}

In [ ]:
files = {
    'PIPELINE_CONFIG' : os.path.join('Tensorflow', 'workspace', 'models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT' : os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME),
    'LABELMAP' : os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME),
}

In [ ]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

# Create train-test

In [ ]:
xmls = glob.glob("Tensorflow/workspace/images/dataset_xml_format/*.xml")
pngs = glob.glob("Tensorflow/workspace/images/dataset_xml_format/*.png")
jpgs = glob.glob("Tensorflow/workspace/images/dataset_xml_format/*.jpg")
imgs = pngs + jpgs

print(len(imgs), len(xmls))

0 0


In [ ]:
os.mkdir('Tensorflow/workspace/images/train')
os.mkdir('Tensorflow/workspace/images/test')

In [ ]:
for img in sorted(imgs)[:len(imgs) // 5]:
    x = img.split('.')[0] + ".xml"
    if x in xmls:
        os.rename(x, x.replace("dataset_xml_format", "test"))
        os.rename(img, img.replace("dataset_xml_format", "test"))
    else:
        print(x)

In [ ]:
for img in sorted(imgs)[len(imgs) // 5:]:
    x = img.split('.')[0] + ".xml"
    if x in xmls:
        os.rename(x, x.replace("dataset_xml_format", "train"))
        os.rename(img, img.replace("dataset_xml_format", "train"))
    else:
        print(x)

In [ ]:
train_xmls = glob.glob("Tensorflow/workspace/images/train/*.xml")
test_xmls = glob.glob("Tensorflow/workspace/images/test/*.xml")

print(len(train_xmls), len(test_xmls))

0 0


# Download Models from TF Model ZOO and install TFOD

In [ ]:
!pip uninstall protobuf matplotlib -y
!pip -q install protobuf 
!pip -q install matplotlib
!pip -q install tensorflow
!pip -q install tensorflow-addons
!pip -q install pyyaml
!pip -q install gin-config
!pip -q install pytz

Found existing installation: protobuf 3.19.6
Uninstalling protobuf-3.19.6:
  Successfully uninstalled protobuf-3.19.6
Found existing installation: matplotlib 3.2.2
Uninstalling matplotlib-3.2.2:
  Successfully uninstalled matplotlib-3.2.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 KB 16.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.12 which is incompatible.
tensorflow-metadata 1.12.0 requires protobuf<4,>=3.13, but you have protobuf 4.21.12 which is incompatible.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.12 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 KB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# https://www.tensorflow.org/install/source_windows

In [ ]:
# !rm -rf '/content/drive/MyDrive/Colab Notebooks/TFODCourse - Copy/Tensorflow/models'

In [ ]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

Cloning into 'Tensorflow/models'...
remote: Enumerating objects: 80657, done.
remote: Counting objects: 100% (184/184), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 80657 (delta 109), reused 164 (delta 96), pack-reused 80473
Receiving objects: 100% (80657/80657), 594.59 MiB | 16.35 MiB/s, done.
Resolving deltas: 100% (57470/57470), done.
Updating files: 100% (3320/3320), done.


In [ ]:
# Install tf object-detection
if os.name == 'posix':
    !apt install protobuf-compiler
    script = f'cd Tensorflow/models/research && \
    protoc object_detection/protos/*.proto --python_out=. && \
    cp object_detection/packages/tf2/setup.py . && \
    python -m pip install .'
    !{script}

if os.name == 'nt':
    url = "https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))
    
    script = f'cd Tensorflow/models/research && \
    protoc object_detection/protos/*.proto --python_out=. && \
    copy object_detection\\packages\\tf2\\setup.py setup.py && \
    python setup.py build && \
    python setup.py install'
    !{script}
    
    !cd Tensorflow/models/research/slim && pip install -e # (--editable) Install a project from a local project path

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.6.1.3-2ubuntu5).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/drive/.shortcut-targets-by-id/11Nbvfv2gDXwVgOrqKSOW69hmt_-zSksQ/TFODCourse - Copy/Tensorflow/models/research
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 90.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 KB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 96.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 KB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 KB 

In [ ]:
VERIFICATION_SCRIPT = os.path.join(
    paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py'
)
# Verify instalation
!{VERIFICATION_SCRIPT}

/bin/bash: Tensorflow/models/research/object_detection/builders/model_builder_tf2_test.py: Permission denied


In [ ]:
if os.name == 'posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME + '.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME + '.tar.gz'}

if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME + '.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME + '.tar.gz'}

--2023-01-21 12:42:02--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.145.128, 2a00:1450:4013:c07::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.145.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.56M  97.8MB/s    in 0.2s    

2023-01-21 12:42:02 (97.8 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]

ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_320x320_coco17_tp

# Create label map

In [ ]:
labels = [
    {'name' : 'drone', 'id' : 1}, 
]

In [ ]:
with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write(f'\tname:\'{label["name"]}\'\n')
        f.write(f'\tid:{label["id"]}\n')
        f.write('}\n')

# Create TFRecords

In [ ]:
# need to correct extensions for xml files:

train_xmls = glob.glob("Tensorflow/workspace/images/train/*.xml")
train_pngs = glob.glob("Tensorflow/workspace/images/train/*.png")
train_jpgs = glob.glob("Tensorflow/workspace/images/train/*.jpg")
train_imgs = train_pngs + train_jpgs

for x in sorted(train_xmls):
    
    if x.split('.')[0] + '.jpg' in train_imgs:
        extension = os.path.split(x.split('.')[0] + '.jpg')[-1]
    if x.split('.')[0] + '.png' in train_imgs:
        extension = os.path.split(x.split('.')[0] + '.png')[-1]
    
#     print(extension)
    
#     print(x)
    with open(x, 'r') as xml_file:
        data = xml_file.read()
        
        
    tree = ET.parse(x)
    root = tree.getroot()
    elem = root.findall('filename')
    for i in elem:
        txt_replace = i.text

    correct_data = data.replace(txt_replace, extension)
    
    with open(x, 'w') as xml_file:
        xml_file.write(correct_data)
        
#     print(data.replace(txt_replace, extension))
#     break

In [ ]:
# Optionally if running in colab
ARCHIVE_FILES = os.path.join(paths['IMAGE_PATH'], 'archive.tar.gz')
if os.path.exists(ARCHIVE_FILES):
    print('tar:')
    !tar -zxvf {ARCHIVE_FILES}

In [ ]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}

In [ ]:
script = f'\
python {files["TF_RECORD_SCRIPT"]} \
-x {os.path.join(paths["IMAGE_PATH"], "train")} \
-l {files["LABELMAP"]} \
-o {os.path.join(paths["ANNOTATION_PATH"], "train.record")}'
!{script}

print(os.path.join(paths["ANNOTATION_PATH"], "train.record"))

Successfully created the TFRecord file: Tensorflow\workspace\annotations\train.record
Tensorflow\workspace\annotations\train.record


In [ ]:
script = f'\
python {files["TF_RECORD_SCRIPT"]} \
-x {os.path.join(paths["IMAGE_PATH"], "test")} \
-l {files["LABELMAP"]} \
-o {os.path.join(paths["ANNOTATION_PATH"], "test.record")}'
!{script}

print(os.path.join(paths["ANNOTATION_PATH"], "test.record"))

Successfully created the TFRecord file: Tensorflow\workspace\annotations\test.record
Tensorflow\workspace\annotations\test.record


# Copy model config to train folder

In [ ]:

# !rm {os.path.join(paths['CHECKPOINT_PATH'], 'pipeline.config')}
# !cp '/content/drive/MyDrive/Colab Notebooks/TFODCourse - Copy/Tensorflow/workspace/models/drone_ssd_mobnet_v3/pipeline.config' {os.path.join(paths['CHECKPOINT_PATH'])}


In [ ]:
if os.name == 'posix':
    !rm {os.path.join(paths['CHECKPOINT_PATH'], 'pipeline.config')}
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

if os.name == 'nt':
    !del {os.path.join(paths['CHECKPOINT_PATH'], 'pipeline.config')}
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

rm: cannot remove 'Tensorflow/workspace/models/drone_ssd_mobnet_v4/pipeline.config': No such file or directory


# Update config for transfer learning

In [ ]:
import tensorflow as tf
import object_detection.utils.config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
model_config = object_detection.utils.config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [ ]:
model_config

{'model': ssd {
   num_classes: 1
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   match

In [ ]:
pipeline_config = object_detection.protos.pipeline_pb2.TrainEvalPipelineConfig()

with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], 'r') as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [ ]:
pipeline_config.model.ssd.num_classes = len(labels)

pipeline_config.train_config.batch_size = BATCH_SIZE

# https://github.com/tensorflow/models/blob/master/research/object_detection/protos/optimizer.proto
# rms_prop_optimizer
# momentum_optimizer
# adam_optimizer
# And then for each optimizer, you can configure the learning rate as one of the following
# constant_learning_rate
# exponential_decay_learning_rate
# manual_step_learning_rate
# cosine_decay_learning_rate


pipeline_config.train_config.optimizer.adam_optimizer.learning_rate.constant_learning_rate.learning_rate = 0.001


# pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.learning_rate_base = 0.05
# pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.warmup_learning_rate = 0.025


pipeline_config.train_config.optimizer.adam_optimizer.learning_rate.exponential_decay_learning_rate.initial_learning_rate = 0.000015
pipeline_config.train_config.optimizer.adam_optimizer.learning_rate.exponential_decay_learning_rate.decay_steps = 3000
pipeline_config.train_config.optimizer.adam_optimizer.learning_rate.exponential_decay_learning_rate.decay_factor = 0.9

# pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.learning_rate_base = 0.05
# pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.warmup_learning_rate = 0.025

pipeline_config.train_config.data_augmentation_options[1].random_crop_image.min_object_covered = 0.25



pipeline_config.train_config.fine_tune_checkpoint = os.path.join(
    paths['PRETRAINED_MODEL_PATH'], 
    PRETRAINED_MODEL_NAME, 
    'checkpoint', 
    'ckpt-0'
)

pipeline_config.train_config.fine_tune_checkpoint_type = "detection"

pipeline_config.train_input_reader.label_map_path= files['LABELMAP']

pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [
    os.path.join(paths['ANNOTATION_PATH'], 'train.record')
]

pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']

pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [
    os.path.join(paths['ANNOTATION_PATH'], 'test.record')
]

In [ ]:
# pipeline_config.train_config.fine_tune_checkpoint = "/content/drive/MyDrive/Colab Notebooks/TFODCourse - Copy/Tensorflow/workspace/models/drone_ssd_mobnet_v3/ckpt-85"

In [ ]:
os.path.join(
    paths['PRETRAINED_MODEL_PATH'], 
    PRETRAINED_MODEL_NAME, 
    'checkpoint', 
    'ckpt-0'
)

'Tensorflow/workspace/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'

In [ ]:
config_text = text_format.MessageToString(pipeline_config)

with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], 'wb') as f:
    f.write(config_text)

In [ ]:
pipeline_config

model {
  ssd {
    num_classes: 1
    image_resizer {
      fixed_shape_resizer {
        height: 320
        width: 320
      }
    }
    feature_extractor {
      type: "ssd_mobilenet_v2_fpn_keras"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 3.9999998989515007e-05
          }
        }
        initializer {
          random_normal_initializer {
            mean: 0.0
            stddev: 0.009999999776482582
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.996999979019165
          scale: true
          epsilon: 0.0010000000474974513
        }
      }
      use_depthwise: true
      override_base_feature_extractor_hyperparams: true
      fpn {
        min_level: 3
        max_level: 7
        additional_layer_depth: 128
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
 

# Train the model

In [ ]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')
print(os.path.exists(TRAINING_SCRIPT))

True


In [ ]:
script = f"python {TRAINING_SCRIPT} \
--model_dir={paths['CHECKPOINT_PATH']} \
--pipeline_config_path={files['PIPELINE_CONFIG']} \
--num_train_steps=30000"

In [ ]:
# run in terminal
print(script)

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/drone_ssd_mobnet_v4 --pipeline_config_path=Tensorflow/workspace/models/drone_ssd_mobnet_v4/pipeline.config --num_train_steps=30000


In [ ]:
!{script}

2023-01-21 14:43:45.921172: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-01-21 14:43:45.921277: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-01-21 14:43:45.921296: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-01-21 14:43:49.624031: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environme

# Evaluate the model

In [ ]:
script = f"python {TRAINING_SCRIPT} \
--model_dir={paths['CHECKPOINT_PATH']} \
--pipeline_config_path={files['PIPELINE_CONFIG']} \
--checkpoint_dir={paths['CHECKPOINT_PATH']}"

In [ ]:
print(script)

python Tensorflow\models\research\object_detection\model_main_tf2.py --model_dir=Tensorflow\workspace\models\new_ssd_mobnet --pipeline_config=Tensorflow\workspace\models\new_ssd_mobnet\pipeline.config --checkpoint_dir=Tensorflow\workspace\models\new_ssd_mobnet


# Load model from checkpoint

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/TFODCourse - Copy/Tensorflow/models/research')

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util, config_util, visualization_utils as viz_utils
from object_detection.builders import model_builder

In [ ]:
# load pipeline config and build model
model_config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=model_config['model'], is_training=False)

In [ ]:
# restore checkpoint
ckpt = tf.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-5')).expect_partial()

In [ ]:
@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# Detect from an image

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [ ]:
IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'test', 'thumbsup.a53e7a1f-86c4-11ec-a413-4cebbd6edeae.jpg')

In [ ]:
os.listdir(os.path.join(paths['IMAGE_PATH'], 'test'))

['livelong.c4207e43-86c4-11ec-9952-4cebbd6edeae.jpg',
 'livelong.c4207e43-86c4-11ec-9952-4cebbd6edeae.xml',
 'thankyou.baf58ad3-86c4-11ec-869a-4cebbd6edeae.jpg',
 'thankyou.baf58ad3-86c4-11ec-869a-4cebbd6edeae.xml',
 'thumbsdown.b09fccf7-86c4-11ec-92a5-4cebbd6edeae.jpg',
 'thumbsdown.b09fccf7-86c4-11ec-92a5-4cebbd6edeae.xml',
 'thumbsup.a53e7a1f-86c4-11ec-a413-4cebbd6edeae.jpg',
 'thumbsup.a53e7a1f-86c4-11ec-a413-4cebbd6edeae.xml']

In [ ]:
img = cv2.imread(IMAGE_PATH)
input_tensor = tf.convert_to_tensor(np.expand_dims(img, 0), dtype=tf.float32)

detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {
    key : value[0, :num_detections].numpy()
    for key, value in detections.items()
}
detections['num_detections'] = num_detections

# detection_classes should be int
detections['detection_classes'] = detections['detection_classes'].astype('int')

label_id_offset = 1
img_detections = img.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
    img_detections,
    detections['detection_boxes'],
    detections['detection_classes'] + label_id_offset,
    detections['detection_scores'],
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=5,
    min_score_thresh=0.7,
    agnostic_mode=False
)

plt.imshow(img_detections[..., ::-1])
plt.show()

# Real time detection from webcam

In [ ]:
import cv2
import tensorflow as tf
import os
import numpy as np

import object_detection
from object_detection.utils import label_map_util, config_util, visualization_utils as viz_utils
from object_detection.builders import model_builder

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# define global variables
CUSTOM_MODEL_NAME = 'new_ssd_mobnet'
# http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = f'http://download.tensorflow.org/models/object_detection/tf2/20200711/{PRETRAINED_MODEL_NAME}.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map_2.pbtxt'
BATCH_SIZE = 4

paths = {
    "WORKSPACE_PATH" : os.path.join('Tensorflow', 'workspace'),
    "SCRIPTS_PATH" : os.path.join('Tensorflow', 'scripts'),
    "APIMODEL_PATH" : os.path.join('Tensorflow', 'models'),
    "ANNOTATION_PATH" : os.path.join('Tensorflow', 'workspace', 'annotations'),
    "IMAGE_PATH" : os.path.join('Tensorflow', 'workspace', 'images'),
    "MODEL_PATH" : os.path.join('Tensorflow', 'workspace', 'models'),
    "PRETRAINED_MODEL_PATH" : os.path.join('Tensorflow', 'workspace', 'pre-trained-models'),
    "CHECKPOINT_PATH" : os.path.join('Tensorflow', 'workspace', 'models', CUSTOM_MODEL_NAME),
    "OUTPUT_PATH" : os.path.join('Tensorflow', 'workspace', 'models', CUSTOM_MODEL_NAME, 'export'),
    "TFJS_PATH" : os.path.join('Tensorflow', 'workspace', 'models', CUSTOM_MODEL_NAME, 'tfjsexport'),
    "TFLITE_PATH" : os.path.join('Tensorflow', 'workspace', 'models', CUSTOM_MODEL_NAME, 'tfliteexport'),
    "PROTOC_PATH" : os.path.join('Tensorflow', 'protoc'),
}
files = {
    'PIPELINE_CONFIG' : os.path.join('Tensorflow', 'workspace', 'models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT' : os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME),
    'LABELMAP' : os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME),
}

category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

# load pipeline config and build model
model_config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=model_config['model'], is_training=False)
# restore checkpoint
ckpt = tf.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-5')).expect_partial()

# open camera
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# get detections
while cap.isOpened():
    ret, frame = cap.read()
    input_tensor = tf.convert_to_tensor(np.expand_dims(frame, 0), dtype=tf.float32)
    
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {
        key : value[0, :num_detections].numpy() for key, value in detections.items()
    }
    detections['num_detections'] = num_detections
    detections['detection_classes'] = detections['detection_classes'].astype('int')

    label_id_offset = 1
    img_detections = frame.copy()
    
    viz_utils.visualize_boxes_and_labels_on_image_array(
        img_detections,
        detections['detection_boxes'],
        detections['detection_classes'] + label_id_offset,
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=10,
        min_score_thresh=0.7,
        agnostic_mode=False
    )
    
    # display image
    cv2.imshow('detector', cv2.resize(img_detections, (width, height)))
    
    # to cancel press "q"
    if cv2.waitKey(10) & 0xff == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

# Save model

In [ ]:
SAVE_MODEL_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'exporter_main_v2.py')
os.path.exists(SAVE_MODEL_SCRIPT)

True

In [ ]:
script = f'python {SAVE_MODEL_SCRIPT} \
--input_type=image_tensor \
--pipeline_config_path={files["PIPELINE_CONFIG"]} \
--trained_checkpoint_dir={paths["CHECKPOINT_PATH"]} \
--output_directory={paths["OUTPUT_PATH"]}'

In [ ]:
print(script)

python Tensorflow/models/research/object_detection/exporter_main_v2.py --input_type=image_tensor --pipeline_config_path=Tensorflow/workspace/models/drone_ssd_mobnet_v4/pipeline.config --trained_checkpoint_dir=Tensorflow/workspace/models/drone_ssd_mobnet_v4 --output_directory=Tensorflow/workspace/models/drone_ssd_mobnet_v4/export


In [ ]:
!{script}

2023-01-21 14:44:31.830506: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-01-21 14:44:31.830616: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-01-21 14:44:31.830634: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-01-21 14:44:35.421529: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environme

# Export to TFJS

In [ ]:
script = f"tensorflowjs_converter \
--input_format=tf_saved_model \
--output_node_names='\
detection_boxes,detection_classes,detection_features,\
detection_multiclass_scores,detection_scores,\
num_detections,raw_detection_boxes,raw_detection_scores' \
--output_format=tfjs_graph_model \
--signature_name=serving_default \
{os.path.join(paths['OUTPUT_PATH'], 'saved_model')} \
{paths['TFJS_PATH']}"

In [ ]:
print(script)

tensorflowjs_converter --input_format=tf_saved_model --output_node_names='detection_boxes,detection_classes,detection_features,detection_multiclass_scores,detection_scores,num_detections,raw_detection_boxes,raw_detection_scores' --output_format=tfjs_graph_model --signature_name=serving_default Tensorflow/workspace/models/drone_ssd_mobnet_v4/export/saved_model Tensorflow/workspace/models/drone_ssd_mobnet_v4/tfjsexport


In [ ]:
!{script}

/bin/bash: tensorflowjs_converter: command not found


# Export to TFLite

In [ ]:
TFLITE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'export_tflite_graph_tf2.py')
os.path.exists(TFLITE_SCRIPT)

True

In [ ]:
script = f"python {TFLITE_SCRIPT} \
--pipeline_config_path={files['PIPELINE_CONFIG']} \
--trained_checkpoint_dir={paths['CHECKPOINT_PATH']} \
--output_directory={paths['TFLITE_PATH']}\
"

In [ ]:
print(script)

python Tensorflow/models/research/object_detection/export_tflite_graph_tf2.py --pipeline_config_path=Tensorflow/workspace/models/drone_ssd_mobnet_v4/pipeline.config --trained_checkpoint_dir=Tensorflow/workspace/models/drone_ssd_mobnet_v4 --output_directory=Tensorflow/workspace/models/drone_ssd_mobnet_v4/tfliteexport


In [ ]:
!{script}

2023-01-21 14:45:22.896110: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-01-21 14:45:22.896211: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-01-21 14:45:22.896230: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-01-21 14:45:26.405375: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environme

In [ ]:
INPUT_MODEL = os.path.join(paths['TFLITE_PATH'], 'saved_model')
TFLITE_MODEL = os.path.join(paths['TFLITE_PATH'], 'saved_model', 'detect.tflite')

In [ ]:
script = f"tflite_convert \
--saved_model_dir={INPUT_MODEL} \
--output_file={TFLITE_MODEL} \
--input_shapes=1,320,320,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays=\
'TFLite_Detection_PostProcess',\
'TFLite_Detection_PostProcess:1',\
'TFLite_Detection_PostProcess:2',\
'TFLite_Detection_PostProcess:3' \
--inference_type=FLOAT \
--allow_custom_ops"

In [ ]:
print(script)

tflite_convert --saved_model_dir=Tensorflow/workspace/models/drone_ssd_mobnet_v4/tfliteexport/saved_model --output_file=Tensorflow/workspace/models/drone_ssd_mobnet_v4/tfliteexport/saved_model/detect.tflite --input_shapes=1,320,320,3 --input_arrays=normalized_input_image_tensor --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' --inference_type=FLOAT --allow_custom_ops


In [ ]:
!{script}

2023-01-21 14:46:02.433187: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-01-21 14:46:02.433286: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-01-21 14:46:02.433304: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-01-21 14:46:05.456968: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environme

# Zip model

In [ ]:
!tar -czf models.tar.gz {paths['CHECKPOINT_PATH']}